In [2]:
from openai import OpenAI
from dotenv import dotenv_values
from pinecone import Pinecone
from tqdm.auto import tqdm
import pickle
import json


c:\Users\suley\Desktop\Projects\edu-advisor-4\edu-advisor\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [7]:
config = dotenv_values("credentials.env")
OPEN_AI_API_KEY = config["openai_api"]
PINECONE_API_KEY = config["Pinecone_api_key"]
PINECONE_ENVIRONMENT = config["Pinecone_environment"]
INDEX_NAME = config["index_name"]

In [8]:
client = OpenAI(api_key= OPEN_AI_API_KEY)

### Convert to Vector

In [12]:
res = client.embeddings.create(
    input=[
        "HI",
    ],
    model="text-embedding-ada-002"
)
print(res.data[0].embedding)

[-0.034233458340168, -0.004512891639024019, -0.022449249401688576, -0.029967444017529488, -0.02886144071817398, 0.029546109959483147, -0.016023892909288406, -0.003703138791024685, 0.008591282181441784, -0.02277841791510582, 0.02015824057161808, -0.009776285849511623, -0.012666380032896996, -0.01432538591325283, 0.007893445901572704, 0.004397682845592499, 0.0407378189265728, -0.01226479560136795, 0.02647826448082924, -0.0009076804271899164, -0.01379871740937233, 0.012969214469194412, -0.004963851533830166, -0.005398353096097708, -0.019328737631440163, -0.0013602862600237131, 0.006899358704686165, -0.021883081644773483, 0.0025494052097201347, -0.03302212059497833, 0.007952695712447166, 0.004078390076756477, -0.03057311289012432, -0.01757756434381008, -0.00945370178669691, -0.02983577735722065, -0.004450349602848291, -0.006682108156383038, 0.013377382420003414, 0.000709356798324734, 0.012962630949914455, -0.005395061802119017, -0.00649777427315712, -0.012699296697974205, -0.02393708936870

### Upload to Vector DB(Pinecone)

In [9]:

# initialize connection to pinecone (get API key at app.pinecone.io)
pc =Pinecone(
    api_key =  PINECONE_API_KEY,
    environment = PINECONE_ENVIRONMENT  #
)
index = pc.Index(INDEX_NAME)


In [ ]:
example_test_data = [
    'hi',
    'hwo are you doing',
    'I am good and you'
]

In [10]:
#DELETE ALL DATA

# returns error if the name space does not exist



# index.delete(delete_all=True, namespace='')

{}

In [4]:
PATHS = ['Vector_Data\iller.pickle','Vector_Data/allUnisMajors.pickle']

informations = []
for path in PATHS:
     with open(path, 'rb') as f:
          temp = pickle.load(f)
          for t in temp:
               informations.append(t)
informations

['ADANA ilinde ADANA BİLİM VE TEKNOLOJİ ÜNİVERSİTESİ, ÇUKUROVA ÜNİVERSİTESİ üniversiteleri bulunaktadır.',
 'ADIYAMAN ilinde ADIYAMAN ÜNİVERSİTESİ üniversiteleri bulunaktadır.',
 'AFYONKARAHİSAR ilinde AFYON KOCATEPE ÜNİVERSİTESİ, AFYONKARAHİSAR SAĞLIK BİLİMLERİ ÜNİVERSİTESİ üniversiteleri bulunaktadır.',
 'AĞRI ilinde AĞRI İBRAHİM ÇEÇEN ÜNİVERSİTESİ üniversiteleri bulunaktadır.',
 'AMASYA ilinde AMASYA ÜNİVERSİTESİ üniversiteleri bulunaktadır.',
 'ANKARA ilinde ANKA TEKNOLOJİ ÜNİVERSİTESİ, ANKARA HACI BAYRAM VELİ ÜNİVERSİTESİ, ANKARA MEDİPOL ÜNİVERSİTESİ, ANKARA MÜZİK VE GÜZEL SANATLAR ÜNİVERSİTESİ, ANKARA SOSYAL BİLİMLER ÜNİVERSİTESİ, ANKARA ÜNİVERSİTESİ, ANKARA YILDIRIM BEYAZIT ÜNİVERSİTESİ, ATILIM ÜNİVERSİTESİ, BAŞKENT ÜNİVERSİTESİ, ÇANKAYA ÜNİVERSİTESİ, GAZİ ÜNİVERSİTESİ, HACETTEPE ÜNİVERSİTESİ, İHSAN DOĞRAMACI BİLKENT ÜNİVERSİTESİ, LOKMAN HEKİM ÜNİVERSİTESİ, ORTA DOĞU TEKNİK ÜNİVERSİTESİ, OSTİM TEKNİK ÜNİVERSİTESİ, TED ÜNİVERSİTESİ, TOBB EKONOMİ VE TEKNOLOJİ ÜNİVERSİTESİ, TÜRK HA

In [12]:
count = 0  # we'll use the count to create unique IDs
batch_size = 1  # process everything in batches of 32

for i in tqdm(range(6685, len(informations), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(informations))
    # get batch of lines and IDs
    lines_batch = informations[i: i+batch_size]
    ids_batch = [str(n) for n in range(i, i_end)]
    # create embeddings
    res = client.embeddings.create(input=lines_batch,model="text-embedding-ada-002")
    embeds = [res.data[0].embedding]
    # prep metadata and upsert batch
    meta = [{'text': line} for line in lines_batch]
    to_upsert = zip(ids_batch, embeds, meta)
    # upsert to Pinecone
    index.upsert(vectors=list(to_upsert))
    count+=1


  0%|          | 0/18632 [00:00<?, ?it/s]

 15%|█▍        | 2757/18632 [29:25<2:49:23,  1.56it/s]


KeyboardInterrupt: 